In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import pandas as pd

In [2]:
file_name = 'data.csv'
headers = ['Price', 'Location', 'Area', 'Street', 'Facade', 'Direction', 'Object', 'Type']
f = open(file_name, "w")   
f.write('\t'.join(headers))
f.write('\n')

1

In [ ]:
def get_info(html_text):
    info = []
    price = html_text.find('div', {'class':'article-price'}).string.strip('\n         ')
    location = html_text.find('div', {'class':'article-info-location'}).string.strip('\n         ')
    details = results[0].find_all('div', {'class':'col-6'})
    details = [detail.text.strip('\n\n               ').replace('---', '') for detail in details]
    info.append(price)
    info.append(location)
    info.extend(details)
    return info

page = 1
while True:
    print(page)
    try:
        url = f'https://meeyland.com/mua-ban-nha-dat/page-{page}'
        print(url)
        html_text = requests.get(url).text
        tree = BeautifulSoup(html_text, 'html.parser')
        results = tree.find_all('div', {'class':'col-sm-6 col-md-4'})
        info = get_info(results[0])
        info = '\t'.join(info)
        f.write(info+'\n')
        if page > 92001:
            break
        page += 1
    except Exception as e:
        print(f'Error: {e}')
        break

    


In [ ]:
f.close()